## Python Blackjack
For this project you will make a Blackjack game using Python. Click <a href="http://www.hitorstand.net/strategy.php">here</a> to familiarize yourself with the the rules of the game. You won't be implementing every rule "down to the letter" with the game, but we will doing a simpler version of the game. This assignment will be given to further test your knowledge on object-oriented programming concepts.

### Rules:

`1. ` The game will have two players: the Dealer and the Player. The game will start off with a deck of 52 cards. The 52 cards will consist of 4 different suits: Clubs, Diamonds, Hearts and Spades. For each suit, there will be cards numbered 1 through 13. <br>
**Note: No wildcards will be used in the program**

`2. ` When the game begins, the dealer will shuffle the deck of cards, making them randomized. After the dealer shuffles, it will deal the player 2 cards and will deal itself 2 cards from. The Player should be able to see both of their own cards, but should only be able to see one of the Dealer's cards.
 
`3. ` The objective of the game is for the Player to count their cards after they're dealt. If they're not satisfied with the number, they have the ability to 'Hit'. A hit allows the dealer to deal the Player one additional card. The Player can hit as many times as they'd like as long as they don't 'Bust'. A bust is when the Player is dealt cards that total more than 21.

`4. ` If the dealer deals the Player cards equal to 21 on the **first** deal, the Player wins. This is referred to as Blackjack. Blackjack is **NOT** the same as getting cards that equal up to 21 after the first deal. Blackjack can only be attained on the first deal.

`5. ` The Player will never see the Dealer's hand until the Player chooses to 'stand'. A Stand is when the player tells the dealer to not deal it anymore cards. Once the player chooses to Stand, the Player and the Dealer will compare their hands. Whoever has the higher number wins. Keep in mind that the Dealer can also bust. 

In [256]:
# game loop
# menu showing current money asking "What would you like to do?"
# 1. Deal 2. Quit
# Dealing asks for a bet and then deals two cards to player two cards to dealer with one hidden
# Wait for player input ask "What would you like to do?" 1. Hit 2. Stand

In [ ]:
import random

class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        return f"{self.rank['rank']} of {self.suit}"

class Deck:
    def __init__(self):
        self.cards = []
        suits = ["spades", "clubs", "hearts", "diamonds"]
        ranks = [
            {"rank": "Ace", "value": 11},
            {"rank": "2", "value": 2},
            {"rank": "3", "value": 3},
            {"rank": "4", "value": 4},
            {"rank": "5", "value": 5},
            {"rank": "6", "value": 6},
            {"rank": "7", "value": 7},
            {"rank": "8", "value": 8},
            {"rank": "9", "value": 9},
            {"rank": "10", "value": 10},
            {"rank": "Jack", "value": 10},
            {"rank": "King", "value": 10},
            {"rank": "Queen", "value": 10},
        ]
        for suit in suits:
            for rank in ranks:
                self.cards.append(Card(suit, rank))

    def shuffle(self):
        if len(self.cards) > 1:
            random.shuffle(self.cards)

    def deal(self, number=1):
        cards_dealt = []
        for i in range(number):
            if len(self.cards) > 0:
                card = self.cards.pop()
                cards_dealt.append(card)
        return cards_dealt

class Hand:
    def __init__(self, dealer=False):
        self.cards = []
        self.value = 0
        self.dealer = dealer

    def add_card(self, card_list):
        self.cards.extend(card_list)

    def calculate_value(self):
        self.value = 0
        has_ace = False

        for card in self.cards:
            card_value = int(card.rank["value"])
            self.value += card_value
            if card.rank["rank"] == "Ace":
                has_ace = True

        if has_ace and self.value > 21:
            self.value -= 10

    def get_value(self):
        self.calculate_value()
        return self.value
    
    def is_blackjack(self):
        return self.get_value() == 21
    
    def display(self, show_all_cards=False):
        print(f'''{"Dealer's" if self.dealer else "Your"} hand:''')
        for index, card in enumerate(self.cards):
            if index == 0 and self.dealer and not show_all_cards and not self.is_blackjack:
                print("-- HIDDEN --")
            else:
                print(card)
        
        if not self.dealer:
            print("Value: ", self.get_value())
        print()

class Game:
    def __init__(self):
        self.cash = 100
        self.bet = 0

    def place_bet(self):
        while True:
            try:
                self.bet = int(input(f"Your cash: ${self.cash}. Place your bet: "))
                if 0 <= self.bet <= self.cash:
                    break
                else:
                    print("Invalid bet amount. Please bet between 0 and your available cash.")
            except ValueError:
                print("Invalid input. Please enter a valid number.")
    
    def play(self):
        deck = Deck()
        deck.shuffle()

        while self.cash > 0:
            player_hand = Hand()
            dealer_hand = Hand(dealer=True)

            self.place_bet()

            for i in range(2):
                player_hand.add_card(deck.deal(1))
                dealer_hand.add_card(deck.deal(1))
            print()
            print("---" * 30)
            player_hand.display()
            dealer_hand.display()

            choice_menu = ""
            while player_hand.get_value() < 21 and choice_menu not in ["stand"]:
                choice_menu = input("Would you like to Hit or Stand?").lower()
                print()
                while choice_menu not in ["hit", "stand"]:
                    choice_menu = input("Would you like to Hit or Stand?").lower()
                    print()
                if choice_menu in ["hit"]:
                    player_hand.add_card(deck.deal(1))
                    player_hand.display()

            if self.check_winner(player_hand, dealer_hand):

                player_hand_value = player_hand.get_value()
                dealer_hand_value = dealer_hand.get_value()

                while dealer_hand_value < 17:
                    dealer_hand.add_card(deck.deal(1))
                    dealer_hand_value = dealer_hand.get_value()

                dealer_hand.display(show_all_cards=True)

                if self.check_winner(player_hand, dealer_hand):
                    result = self.check_winner(player_hand, dealer_hand, True)

                    if result == "Player wins":
                        self.cash += self.bet
                    elif result == "Dealer wins":
                        self.cash -= self.bet

            print(f"\nYou now have ${self.cash}")

    def check_winner(self, player_hand, dealer_hand, game_over=False):
        if not game_over:
            if player_hand.get_value() > 21:
                print("BUST. Dealer wins.")
                self.cash -= self.bet
                return "Dealer wins"
            elif dealer_hand.get_value() > 21:
                print("Dealer BUSTED. You win!")
                self.cash += self.bet
                return "Player wins"
            elif dealer_hand.is_blackjack() and player_hand.is_blackjack():
                print("Both players have blackjack, TIE!")
                return "Tie"
            elif player_hand.is_blackjack():
                print("BLACKJACK! You win!")
                self.cash += self.bet
                return "Player wins"
            elif dealer_hand.is_blackjack():
                print("Dealer has Blackjack. You lose.")
                self.cash -= self.bet
                return "Dealer wins"
        else:
            if player_hand.get_value() > dealer_hand.get_value():
                print("You win!")
                self.cash += self.bet
                return "Player wins"
            elif player_hand.get_value() == dealer_hand.get_value():
                print("Tie!")
                return "Tie"
            else:
                print("Dealer wins.")
                self.cash -= self.bet
                return "Dealer wins"
        return "Tie"

game = Game()
game.play()
